# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [37]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import time
from datetime import date
%matplotlib inline

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [38]:
city_df_path = "output_data/city-df.csv"


# Read the mouse data and the study results
city_data = pd.read_csv(city_df_path)
city_data


,City,Country,Date,Latitude,Longitude,Humidity,Temperature,Temp Max,Cloudiness,Wind Speed
0,langsa,ID,1.624088e+09,4.4683,97.9683,68.0,84.51,84.51,78.0,7.54
1,vallenar,CL,1.624088e+09,-28.5708,-70.7581,60.0,52.68,52.68,0.0,3.44
2,ijaki,US,1.624087e+09,71.2906,-156.7887,71.0,48.22,48.22,75.0,5.75
3,barrow,NZ,1.624087e+09,-46.1927,168.8643,96.0,41.49,41.49,14.0,3.71
4,mataura,MU,1.624087e+09,-20.4081,57.7000,69.0,77.29,77.29,40.0,6.91
...,...,...,...,...,...,...,...,...,...,...
539,ndele,GB,1.624088e+09,56.0380,-5.4321,87.0,56.01,57.18,100.0,1.81
540,shibarghan,AU,1.624088e+09,-13.8483,136.4192,64.0,73.27,73.27,85.0,8.05
541,concepcion,SA,1.624088e+09,25.0213,37.2685,61.0,85.50,85.50,0.0,14.20
542,sinnamary,RU,1.624088e+09,45.2333,147.8833,70.0,48.45,48.45,19.0,6.20


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [39]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)


In [40]:
# Store 'Lat' and 'Lng' into  locations 
locations = city_data[["Latitude", "Longitude"]].astype(float)

# Convert Poverty Rate to float and store
# HINT: be sure to handle NaN values
city_data = city_data

humidity = city_data["Humidity"].astype(float)

In [42]:
# Create a poverty Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius = 1)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [43]:
city_ideal = city_data.loc[(city_data["Temperature"] < 80) & (city_data["Temperature"] > 70) & (city_data["Wind Speed"] < 10) & (city_data["Cloudiness"] == 0)]
city_ideal

,City,Country,Date,Latitude,Longitude,Humidity,Temperature,Temp Max,Cloudiness,Wind Speed
40,severo-kurilsk,RU,1.624087e+09,59.7035,30.7861,45.0,79.23,80.80,0.0,8.01
109,nador,FR,1.624087e+09,48.6252,6.3497,61.0,76.62,76.98,0.0,2.30
155,jutai,TR,1.624088e+09,38.6908,35.5538,43.0,72.27,72.27,0.0,3.44
194,creel,RU,1.624088e+09,59.7775,30.8181,45.0,78.89,80.96,0.0,8.32
243,lompoc,US,1.624088e+09,28.0170,-97.5169,81.0,77.52,82.83,0.0,5.39
439,padang,LY,1.624088e+09,31.7561,25.0865,62.0,74.79,74.79,0.0,9.73
471,balabac,US,1.624088e+09,33.3582,-94.2544,83.0,71.78,73.69,0.0,4.34
521,marzuq,SA,1.624088e+09,31.6725,38.6637,38.0,79.05,79.05,0.0,6.91
533,kamaishi,TR,1.624088e+09,39.6381,34.4672,34.0,77.04,77.04,0.0,3.40
543,anloga,AU,1.624088e+09,-20.7377,116.8463,38.0,75.42,75.42,0.0,9.51


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [48]:
hotel_df = city_ideal.copy()
hotel_df['Hotel Name'] = ""

hotel_df

,City,Country,Date,Latitude,Longitude,Humidity,Temperature,Temp Max,Cloudiness,Wind Speed,Hotel Name
40,severo-kurilsk,RU,1.624087e+09,59.7035,30.7861,45.0,79.23,80.80,0.0,8.01,
109,nador,FR,1.624087e+09,48.6252,6.3497,61.0,76.62,76.98,0.0,2.30,
155,jutai,TR,1.624088e+09,38.6908,35.5538,43.0,72.27,72.27,0.0,3.44,
194,creel,RU,1.624088e+09,59.7775,30.8181,45.0,78.89,80.96,0.0,8.32,
243,lompoc,US,1.624088e+09,28.0170,-97.5169,81.0,77.52,82.83,0.0,5.39,
439,padang,LY,1.624088e+09,31.7561,25.0865,62.0,74.79,74.79,0.0,9.73,
471,balabac,US,1.624088e+09,33.3582,-94.2544,83.0,71.78,73.69,0.0,4.34,
521,marzuq,SA,1.624088e+09,31.6725,38.6637,38.0,79.05,79.05,0.0,6.91,
533,kamaishi,TR,1.624088e+09,39.6381,34.4672,34.0,77.04,77.04,0.0,3.40,
543,anloga,AU,1.624088e+09,-20.7377,116.8463,38.0,75.42,75.42,0.0,9.51,


In [49]:
#Set parameters to search for hotels with 5000 meters.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [55]:
for index, row in hotel_df.iterrows():
    lat = row["Latitude"]
    lng = row["Longitude"]
    city_name = row["City"]
    
    params["location"] = f"{lat},{lng}"
    
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    results = response['results']
    
    #Save the hotel name to the dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
    except (KeyError, IndexError):
        print("Could not find result")
        print("-------------------------")
        #delay the code by one second to avoid an error
        time.sleep(1)
    print("--------------------Ending Search")

Retrieving Results for Index 40: severo-kurilsk.
Could not find result
-------------------------
Retrieving Results for Index 109: nador.
Could not find result
-------------------------
Retrieving Results for Index 155: jutai.
Closest hotel in jutai is Radisson Blu Hotel Kayseri.
Retrieving Results for Index 194: creel.
Closest hotel in creel is Hotel in Otradnoe.
Retrieving Results for Index 243: lompoc.
Closest hotel in lompoc is Motel 6 Sinton, TX.
Retrieving Results for Index 439: padang.
Closest hotel in padang is Hotel Burdy.
Retrieving Results for Index 471: balabac.
Could not find result
-------------------------
Retrieving Results for Index 521: marzuq.
Closest hotel in marzuq is Golden Dune Hotel.
Retrieving Results for Index 533: kamaishi.
Closest hotel in kamaishi is Mustafas Hotel.
Retrieving Results for Index 543: anloga.
Closest hotel in anloga is Karratha International Hotel.


In [51]:
hotel_df

,City,Country,Date,Latitude,Longitude,Humidity,Temperature,Temp Max,Cloudiness,Wind Speed,Hotel Name
40,severo-kurilsk,RU,1.624087e+09,59.7035,30.7861,45.0,79.23,80.80,0.0,8.01,
109,nador,FR,1.624087e+09,48.6252,6.3497,61.0,76.62,76.98,0.0,2.30,
155,jutai,TR,1.624088e+09,38.6908,35.5538,43.0,72.27,72.27,0.0,3.44,Radisson Blu Hotel Kayseri
194,creel,RU,1.624088e+09,59.7775,30.8181,45.0,78.89,80.96,0.0,8.32,
243,lompoc,US,1.624088e+09,28.0170,-97.5169,81.0,77.52,82.83,0.0,5.39,"Motel 6 Sinton, TX"
439,padang,LY,1.624088e+09,31.7561,25.0865,62.0,74.79,74.79,0.0,9.73,Hotel Burdy
471,balabac,US,1.624088e+09,33.3582,-94.2544,83.0,71.78,73.69,0.0,4.34,
521,marzuq,SA,1.624088e+09,31.6725,38.6637,38.0,79.05,79.05,0.0,6.91,Golden Dune Hotel
533,kamaishi,TR,1.624088e+09,39.6381,34.4672,34.0,77.04,77.04,0.0,3.40,Mustafas Hotel
543,anloga,AU,1.624088e+09,-20.7377,116.8463,38.0,75.42,75.42,0.0,9.51,Latitude20 The Dunes Apartments


In [53]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [54]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)
# Display figure
fig


Figure(layout=FigureLayout(height='420px'))